In [ ]:
import pandas as pd
import json
import re
from tqdm.auto import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)


lines = []
for i in range(1,10):
    with open(f"/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/wiki-pages/wiki-00{i}.jsonl", encoding="utf8") as f:
        lines.extend(f.read().splitlines())
for i in range(10,25):
    with open(f"/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/wiki-pages/wiki-0{i}.jsonl", encoding="utf8") as f:
        lines.extend(f.read().splitlines())

wiki_df_0 = pd.DataFrame([json.loads(line) for line in lines])

wiki_text = []
lines = wiki_df_0['lines'].to_list()
ids = wiki_df_0['id'].to_list()

for i in range(len(lines)):
    for j in re.split(r"\n(?=[0-9])", lines[i]):
        j_list = j.split('\t')
        if len(j_list)>=2 and j_list[1]!='':
            wiki_text.append([ids[i], j_list[0], j_list[1]])

wiki_df = pd.DataFrame(wiki_text, columns=['id', 'line', 'text'])

lines = []
with open(f"/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/public/public_train.jsonl", encoding="utf8") as f:
    lines.extend(f.read().splitlines())
train_df = pd.DataFrame([json.loads(line) for line in lines])

lines = []
with open(f"/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/public/public_test.jsonl", encoding="utf8") as f:
    lines.extend(f.read().splitlines())
test_df = pd.DataFrame([json.loads(line) for line in lines])


In [9]:
tokenized_sent = pd.read_csv('/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/tokenized_sent.csv')['0'].apply(lambda x: str(x).split()).tolist()
tokenized_doc = pd.read_csv('/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/tokenized_doc.csv')['0'].apply(lambda x: str(x).split()).tolist()
tokenized_query = pd.read_csv('/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/tokenized_claim.csv')['0'].apply(lambda x: str(x).split()).tolist()
tokenized_query_test = pd.read_csv('/nfs/nas-6.1/wclu/AICUP/2023/aicup-2023-nlp/data/tokenized_claim_private.csv')['0'].apply(lambda x: str(x).split()).tolist()

In [10]:
wiki_df_0['doc'] = tokenized_doc
wiki_df['sent'] = tokenized_sent
train_df['tokenized_query'] = tokenized_query
test_df['tokenized_query'] = tokenized_query_test

In [11]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_doc)

In [12]:
NUM_DOC = 10
def predict_doc(query):
    doc_scores = bm25.get_scores(query)
    predict = wiki_df_0['id'].iloc[doc_scores.argsort()[::-1][:NUM_DOC]].tolist()
    return predict

#train_df['predicted_doc'] = train_df['tokenized_query'].parallel_apply(predict_doc)

test_df['predicted_doc'] = test_df['tokenized_query'].parallel_apply(predict_doc)

In [15]:
test_df.to_pickle('data/test_private_doc100.pkl')

In [16]:
test_df

,id,claim,tokenized_query,predicted_doc
0,21498,雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與...,"[雞形目, 的, 鳥, 腿腳, 強健, ，, 擅長, 在, 地面, 奔跑, ，, 其中, 有...","[雞形目, 斯卡利亞壯麗鳥, 平腹蛛科, 非洲刺毛鼠屬, 耳烏賊目, 肉座菌目, 紅豆杉科,..."
1,13037,教會剛建立時為解決內部的一些問題，使徒們寫下許多便條，其中有八卷不是保羅寫的為大公書信。,"[教會, 剛, 建立, 時, 為, 解決, 內部, 的, 一些, 問題, ，, 使徒, 們,...","[新約書信, 大公書信, 提多書, 歌羅西書, 羅馬書, 書信_(文體), 哥林多前書, 保..."
2,18652,羅伯·昆蘭於明尼蘇達州聖保羅市出生。,"[羅伯, ·, 昆蘭, 於, 明尼蘇達州, 聖保羅市, 出生, 。]","[羅伯·昆蘭, 基斯·高文_(政治人物), 聖保羅機場, 聖保羅國際機場, 聖保羅市, 羅伯..."
3,21378,2015年美國網球公開賽女子單打比賽裡小威廉絲是上一屆的冠軍。,"[2015年, 美國, 網球, 公開賽, 女子, 單打, 比賽, 裡, 小, 威廉絲, 是,...","[2015年溫布頓網球錦標賽女子單打比賽, 2015年美國網球公開賽, 2016年美國網球公..."
4,18624,南陽郡的地方豪族出身的漢光武帝劉秀為太祖劉邦的九世孫。,"[南陽郡, 的, 地方, 豪族, 出身, 的, 漢, 光武帝, 劉秀, 為, 太祖, 劉邦,...","[漢光武帝, 樊嫺都, 劉讓_(臨邑侯), 劉欽_(南頓縣令), 更始帝, 劉信_(汝陰侯)..."
...,...,...,...,...
8044,12412,劉德華在中國星集團拍攝的最後一部電影爲大隻佬_(電影），之後他就結束與中國星集團14年賓主關係。,"[劉德華, 在, 中國, 星, 集團, 拍攝, 的, 最後, 一, 部, 電影, 爲, 大隻...","[最愛女人購物狂, 大隻佬_(電影), 星劇本, 華夏電影, 賭城風雲III, 女子監獄, ..."
8045,9611,視光學是利用光學儀器搭配侵入的方式來治療患者雙眼視覺異常。,"[視光學, 是, 利用, 光學, 儀器, 搭配, 侵入, 的, 方式, 來, 治療, 患者,...","[視光學, 光學工程, 雙眼視覺, 耳面症候群, 立體視覺, 骨髓移植, 儀器飛行, 不可能..."
8046,10538,婊子千層麪這首歌諷刺了印度音樂唱片公司 「 T Series 」 ， 用來回應T Serie...,"[婊子, 千, 層, 麪, 這, 首, 歌, 諷刺, 了, 印度, 音樂, 唱片, 公司, ...","[婊子千層麪, PewDiePie與TSeries之爭, PewDiePie與T-Serie..."
8047,11757,兩棲動物由於在水中適應比不上腔棘魚，在陸地生存又落後羊膜動物，因此現今大多種類已絕種。,"[兩棲, 動物, 由於, 在, 水, 中, 適應, 比不上, 腔棘魚, ，, 在, 陸地, ...","[兩棲動物, 羊膜動物, 合弓綱, 水生動物, 石炭蜥目, 巴基鯨科, 四足動物, 節胸屬,..."


In [ ]:
def evi_doc(evidence_list):
    if evidence_list[1] == None:
        return None
    e_list = []
    for evidence in evidence_list[0]:
        id = evidence[2]
        if not(id in e_list):
            e_list.append(id)
    return e_list

train_df_info['evidence_doc'] = train_df_info['evidence'].apply(evi_doc)

def evi_sent(evidence_list):
    if evidence_list[1] == None:
        return None
    e_list = []
    for evidence in evidence_list[0]:
        id = evidence[2]
        line = evidence[3]
        e_list.append(f'{id}_{line}')
    return e_list

train_df_info['evidence_line'] = train_df_info['evidence'].apply(evi_sent)

In [ ]:
score = []
total = len(train_df_info)
for NUM_PAGE in tqdm(range(NUM_DOC)):
    wrong = 0
    for i in range(total):
        for e in train_df_info['evidence_list'][i]:
            if not(e in train_df_info['predicted_pages'][i][:NUM_PAGE]):
                wrong += 1
                break
    score.append(round((total-wrong)/total, 3))

In [ ]:
for i in range(100):
    print(f'{i}_{score[i]}')

In [ ]:
def sent_from_doc(docs):
    sents = []
    ids = []
    for doc in docs:
        ind = (wiki_df['id']==doc)
        sents.extend(wiki_df[ind]['sent'].tolist())
        id = wiki_df[ind]['id'].astype('str')
        line = wiki_df[ind]['line'].astype('str')
        ids.extend((id+'_'+line).tolist())
    return sents, ids

def rank_sent(query_sents_ids):
    bm25 = BM25Okapi(query_sents_ids[1])
    predict = bm25.get_top_n(query_sents_ids[0], query_sents_ids[2], n=len(query_sents_ids[2]))
    return predict



In [ ]:
train_df_info['predicted_sents'] = train_df_info['predicted_pages'].parallel_apply(sent_from_doc)

In [ ]:
train_df_info = pd.read_pickle('data/train_df_info_doc2sent.pkl')

In [ ]:
query_sents_ids = []
for i in tqdm(range(len(train_df_info))):
    query_sents_ids.append([train_df_info['tokenized_query'][i], train_df_info['predicted_sents'][i][0], train_df_info['predicted_sents'][i][1]])

In [ ]:
train_df_info['rerank_sents'] = pd.Series(query_sents_ids).parallel_apply(rank_sent)

In [ ]:
score = []
total = len(train_df_info)
NUM_SENT=1000
for num_sent in tqdm(range(NUM_SENT)):
    wrong = 0
    for i in range(total):
        l = len(train_df_info['rerank_sents'][i])
        for e in train_df_info['evidence_line'][i]:
            m = min(num_sent, l)
            if not(e in train_df_info['rerank_sents'][i][:m]):
                wrong += 1
                break
    score.append(round((total-wrong)/total, 3))

In [ ]:
for i in range(500):
    print(f'{i}_{score[i]}')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(score)

In [ ]:
import pandas as pd
train_df = pd.read_pickle('data/train_df_doc100.pkl')

In [ ]:
def evi_doc(evidence_list):
    if len(evidence_list[0])>1 and evidence_list[0][1] == None:
        return None
    e_list = []
    for evidence in evidence_list[0]:
        id = evidence[2]
        if not(id in e_list):
            e_list.append(id)
    return e_list

train_df['ground_doc'] = train_df['evidence'].apply(evi_doc)

def evi_sent(evidence_list):
    if len(evidence_list[0])>1 and evidence_list[0][1] == None:
        return None
    e_list = []
    for evidence in evidence_list[0]:
        id = evidence[2]
        line = evidence[3]
        e_list.append(f'{id}_{line}')
    return e_list

train_df['ground_sent'] = train_df['evidence'].apply(evi_sent)

In [ ]:
train_df[['id','label','claim','evidence','ground_doc','ground_sent', 'predicted_pages']].to_pickle('data/train_doc100.pkl')